# Binary model trained on Norwegian Data

## Model

In [7]:
import os, sys
from pathlib import Path

# Finn repo-root ved å gå oppover til vi finner src/train.py
here = Path().cwd().resolve()
PROJECT_ROOT = None
for p in [here, *here.parents]:
    if (p / "src" / "train.py").exists():
        PROJECT_ROOT = p
        break

assert PROJECT_ROOT is not None, f"Fant ikke src/train.py fra {here}"

os.chdir(PROJECT_ROOT)
sys.path.insert(0, str(PROJECT_ROOT))

print("Repo root:", PROJECT_ROOT)
print("Train path:", PROJECT_ROOT / "src" / "train.py")


Repo root: /cluster/home/larshfle/superpoint_transformer_new
Train path: /cluster/home/larshfle/superpoint_transformer_new/src/train.py


In [ ]:
import importlib
m = importlib.import_module("src.datamodules.norway_binary")
print("ok import", m)
print("has class", hasattr(m, "NorwayBinaryDataModule"))


In [ ]:
### Fungerende kommando
"""
source ~/.bashrc && conda activate spta100 && cd /cluster/home/larshfle/superpoint_transformer && python src/train.py \
    experiment=semantic/dales_11g \
    datamodule=semantic/norway_binary \
    paths.data_dir=/cluster/home/larshfle/datasets/norway_binary_testing \
    trainer.max_epochs=20 \
    trainer.check_val_every_n_epoch=5 \
    callbacks.model_checkpoint.save_last=True \
    callbacks.model_checkpoint.every_n_epochs=5 \
    callbacks.model_checkpoint.monitor=null \
    test=True \
    logger=csv 2>&1 | tee /tmp/train_output.log
"""



# Med Optuna hyperparameter tuning
"""
source ~/.bashrc && conda activate spta100 && cd /cluster/home/larshfle/superpoint_transformer && python src/train.py -m \
    hparams_search=norway_binary_optuna \
    experiment=semantic/dales_11g \
    datamodule=semantic/norway_binary \
    paths.data_dir=/cluster/home/larshfle/datasets/norway_binary_testing \
    trainer.max_epochs=20 \
    trainer.check_val_every_n_epoch=5 \
    callbacks.model_checkpoint.save_last=True \
    callbacks.model_checkpoint.every_n_epochs=5 \
    callbacks.model_checkpoint.monitor=null \
    test=False \
    logger=csv 2>&1 | tee /tmp/optuna_search.log
"""

#Enkel test
"""
source ~/.bashrc && conda activate spt && cd /cluster/home/larshfle/superpoint_transformer_new && python src/train.py \
    experiment=semantic/dales_11g \
    datamodule=semantic/norway_binary \
    paths.data_dir=/cluster/home/larshfle/datasets/norway_binary_testing \
    trainer.max_epochs=50 \
    trainer.check_val_every_n_epoch=5 \
    callbacks.model_checkpoint.save_last=True \
    callbacks.model_checkpoint.every_n_epochs=5 \
    callbacks.model_checkpoint.monitor=null \
    test=True \
    logger=csv 2>&1 | tee /tmp/train_output.log
"""


#Slette gamle processed data
""" 
rm -rf /cluster/home/larshfle/datasets/norway_binary_testing/norwaybinaryals/processed/
"""

In [ ]:
import subprocess, sys, os
from pathlib import Path

os.environ["HYDRA_FULL_ERROR"] = "1"
os.environ["WANDB_MODE"] = "disabled"
os.environ["WANDB_SILENT"] = "true"

DATA_ROOT = Path("/cluster/home/larshfle/datasets/norway_binary_testing").resolve()
TRAIN_PY = (Path().cwd() / "src" / "train.py").resolve()

cmd = [
    sys.executable, str(TRAIN_PY),
    "experiment=semantic/dales_11g",
    "datamodule=semantic/norway_binary",
    f"paths.data_dir={DATA_ROOT}",
    "trainer.max_epochs=1",
    "+trainer.limit_train_batches=10",
    "+trainer.limit_val_batches=2",
    "+trainer.num_sanity_val_steps=0",
    "+trainer.enable_progress_bar=true",
    "logger=csv",
]

print(" ".join(cmd))

p = subprocess.run(cmd, cwd=str(Path().cwd()), text=True, capture_output=True)
print("RETURN CODE:", p.returncode)
print("\n===== STDOUT =====\n", p.stdout[-8000:])  # viser siste del
print("\n===== STDERR =====\n", p.stderr[-8000:])  # ofte der feilen ligger

# Hvis du vil stoppe notebooken når den feiler:
p.check_returncode()


In [ ]:
"""
python src/train.py \
    experiment=semantic/dales_11g \
    datamodule=semantic/norway_binary \
    paths.data_dir=/cluster/home/larshfle/datasets/norway_binary_testing \
    trainer.max_epochs=1 \
    +trainer.limit_train_batches=10 \
    +trainer.limit_val_batches=2 \
    logger=csv
"""

In [ ]:
import os
os.environ["HYDRA_FULL_ERROR"] = "1"
os.environ["WANDB_MODE"] = "disabled"
os.environ["WANDB_SILENT"] = "true"

DATA_ROOT = "/cluster/home/larshfle/datasets/norway_binary_testing"

# Kjør trening - UTEN limit_batches for å få checkpoint lagret
!python src/train.py \
    experiment=semantic/dales_11g \
    datamodule=semantic/norway_binary \
    paths.data_dir={DATA_ROOT} \
    trainer.max_epochs=1 \
    callbacks.model_checkpoint.save_last=True \
    callbacks.model_checkpoint.every_n_epochs=1 \
    callbacks.model_checkpoint.monitor=null \
    logger=csv

## Evaluering på en tile

In [ ]:
# Finn siste checkpoint
from pathlib import Path
import glob

logs_dir = Path("logs/train/runs")
runs = sorted(logs_dir.glob("*"), key=lambda x: x.name, reverse=True)

ckpt_path = None
for run in runs:
    ckpts = list(run.glob("checkpoints/*.ckpt"))
    if ckpts:
        ckpt_path = sorted(ckpts, key=lambda x: x.stat().st_mtime, reverse=True)[0]
        print(f"Fant checkpoint: {ckpt_path}")
        break

if ckpt_path is None:
    print("⚠️  Ingen checkpoint funnet!")
    print("Tips: Kjør trening med flere epochs eller uten limit_train_batches")
    print("\nSiste runs (uten checkpoint):")
    for run in runs[:5]:
        print(f"  - {run.name}: {list(run.iterdir())}")

## Visuals

In [4]:
# Visualiser treningsresultater fra CSV-loggene
import pandas as pd
from pathlib import Path

# Finn siste run med metrics - sjekk flere mulige stier
possible_paths = [
    Path("logs/train/runs"),
    Path("/cluster/home/larshfle/superpoint_transformer_new/logs/train/runs"),
]

metrics_file = None
for logs_dir in possible_paths:
    if logs_dir.exists():
        runs = sorted(logs_dir.glob("*/csv/version_0/metrics.csv"), key=lambda x: x.parent.parent.parent.name, reverse=True)
        if runs:
            metrics_file = runs[0]
            break

if metrics_file and metrics_file.exists():
    print(f"Leser metrics fra: {metrics_file.parent.parent.parent.name}")
    
    df = pd.read_csv(metrics_file)
    
    # Vis siste resultater
    print("\n" + "="*50)
    print("RESULTATER")
    print("="*50)
    
    # Validation metrics
    val_cols = [c for c in df.columns if c.startswith('val/')]
    if val_cols:
        print("\n📊 Validation:")
        for col in val_cols:
            vals = df[col].dropna()
            if not vals.empty:
                val = vals.iloc[-1]
                print(f"  {col.replace('val/', '')}: {val:.2f}")
    
    # Test metrics
    test_cols = [c for c in df.columns if c.startswith('test/')]
    if test_cols:
        print("\n🧪 Test:")
        for col in test_cols:
            vals = df[col].dropna()
            if not vals.empty:
                val = vals.iloc[-1]
                print(f"  {col.replace('test/', '')}: {val:.2f}")
    
    # Train metrics
    train_cols = [c for c in df.columns if c.startswith('train/')]
    if train_cols:
        print("\n🏋️ Train:")
        for col in train_cols:
            vals = df[col].dropna()
            if not vals.empty:
                val = vals.iloc[-1]
                print(f"  {col.replace('train/', '')}: {val:.2f}")
else:
    print("⚠️ Ingen metrics funnet!")
    print("Tips: Sjekk at du har kjørt trening og at logs/train/runs/ inneholder resultater")

Leser metrics fra: 2026-02-04_09-30-42

RESULTATER

📊 Validation:
  iou_ground: 58.04
  iou_not_ground: 62.69
  loss: 10.46
  macc: 76.00
  macc_best: 76.04
  miou: 60.37
  miou_best: 60.41
  oa: 75.39
  oa_best: 75.42

🧪 Test:
  iou_ground: 58.05
  iou_not_ground: 62.70
  loss: 10.46
  macc: 76.00
  miou: 60.37
  oa: 75.40

🏋️ Train:
  iou_ground: 66.32
  iou_not_ground: 68.29
  loss: 9.90
  macc: 80.49
  miou: 67.30
  oa: 80.48


In [ ]:
# Last modell og kjør prediksjon på en tile
import torch
from src.utils import init_config
from src.datasets.norway_binary import read_norway_laz
from src.datasets.norway_binary_config import CLASS_NAMES, CLASS_COLORS
import hydra

# Stier
CKPT_PATH = "/cluster/home/larshfle/superpoint_transformer_new/logs/train/runs/2026-02-02_14-20-56/checkpoints/last.ckpt"
DATA_ROOT = "/cluster/home/larshfle/datasets/norway_binary_testing"
TILE_PATH = f"{DATA_ROOT}/norwaybinaryals/raw/test/32-1-468-145-52.laz"

# Last konfig
cfg = init_config(
    overrides=[
        "experiment=semantic/dales_11g",
        "datamodule=semantic/norway_binary",
        f"paths.data_dir={DATA_ROOT}",
    ]
)

# Instantier datamodule og modell
datamodule = hydra.utils.instantiate(cfg.datamodule)
model = hydra.utils.instantiate(cfg.model)

# Last checkpoint
print(f"Laster checkpoint: {CKPT_PATH}")
ckpt = torch.load(CKPT_PATH, map_location='cuda')
model.load_state_dict(ckpt['state_dict'])
model = model.cuda().eval()
print("Modell lastet!")

Laster checkpoint: /cluster/home/larshfle/superpoint_transformer_new/logs/train/runs/2026-02-02-14_20-56/checkpoints/last.ckpt


FileNotFoundError: [Errno 2] No such file or directory: '/cluster/home/larshfle/superpoint_transformer_new/logs/train/runs/2026-02-02-14_20-56/checkpoints/last.ckpt'

In [ ]:
# Les tile og kjør prediksjon
print(f"Leser tile: {TILE_PATH}")
data = read_norway_laz(TILE_PATH)
print(f"Antall punkter: {data.pos.shape[0]}")

# Preprocessing
nag = datamodule.pre_transform(data)
nag = nag.cuda()
nag = datamodule.on_device_val_transform(nag)

# Prediksjon
with torch.no_grad():
    output = model(nag)

# Legg til prediksjoner i NAG-objektet for visualisering
nag.semantic_pred = output.semantic_pred
print("Prediksjon ferdig!")

In [ ]:
# Visualiser med innebygd show() - viser Labels, Predictions og Error mode
from src.visualization import show

# Legg til prediksjonene på NAG-en slik at "Semantic" modus fungerer
# Bruk voxel_semantic_pred med super_index for å mappe predictions til superpoints
nag[0].semantic_pred = output.voxel_semantic_pred(super_index=nag[0].super_index)

print(f"NAG har {nag.num_levels} nivåer")
print(f"Level 0: {nag[0].num_points} superpoints")
print(f"semantic_pred shape: {nag[0].semantic_pred.shape}")
print(f"Unike predictions: {nag[0].semantic_pred.unique().tolist()}")
print(f"Labels (y) unike: {nag[0].y.unique().tolist()}")

# Vis NAG med alle moduser (Labels, Semantic Pred, Error, osv.)
nag.show(
    class_names=CLASS_NAMES,
    class_colors=CLASS_COLORS,
    num_classes=2,
    radius=100,
    max_points=50000,
    centroids=True,
    h_edge=True,
    title="Norway Binary - Predictions vs Ground Truth"
)

In [ ]:
# Eksporter visualisering som HTML-fil (kan åpnes i nettleser)
nag.show(
    class_names=CLASS_NAMES,
    class_colors=CLASS_COLORS,
    num_classes=2,
    radius=100,
    max_points=50000,
    path="norway_binary_visualization.html",  # lagrer som HTML
    title="Norway Binary Model Results"
)
print("Visualisering lagret til: norway_binary_visualization.html")